# Piotroski score through Yahoo finance by web scraping the financials

In [1]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [34]:
def statement(ticker,statement='IS',period='A'):
    if statement == 'IS' and period == 'A':
        temp = 'incomeStatementHistory'
        temp1 = 'incomeStatementHistory'
    elif statement == 'IS' and period == 'Q':
        temp = 'incomeStatementHistoryQuarterly'
        temp1 = 'incomeStatementHistory'
    elif statement == 'BS' and period == 'A':
        temp = 'balanceSheetHistory'
        temp1 = 'balanceSheetStatements'
    elif statement == 'BS' and period == 'Q':
        temp = 'balanceSheetHistoryQuarterly'
        temp1 = 'balanceSheetStatements'
    elif statement == 'CF' and period == 'A':
        temp = 'cashflowStatementHistory'
        temp1 = 'cashflowStatements'
    elif statement == 'CF' and period == 'Q':
        temp = 'cashflowStatementHistoryQuarterly'
        temp1 = 'cashflowStatements'
    
    url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'
    response = requests.get(url_financials.format(ticker, ticker))
    soup = BeautifulSoup(response.text, 'html.parser')
    pattern = re.compile(r'\s--\sData\s--\s')
    script_data = soup.find('script', text=pattern).contents[0]
    # find the starting position of the json string
    start = script_data.find("context")-2
    # slice the json string
    json_data = json.loads(script_data[start:-12])
    state_ment = json_data['context']['dispatcher']['stores']['QuoteSummaryStore'][temp][temp1]
    data = []
    for s in state_ment:
        statement = {}
        for key, val in s.items():
            try:
                statement[key] = val['raw']
            except TypeError:
                continue
            except KeyError:
                continue
        data.append(statement)
    data = data[:4]
    df = pd.DataFrame(data).T
    df.columns = ['1', '2', '3', '4']
    return df

In [40]:
def net_income(ticker):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    return float(df.loc['netIncome'][0])

def roa(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['totalAssets'][0]),float(df.loc['totalAssets'][1])
    av_assets=(p+q)/2
    return net_income(ticker)/av_assets

def ocf(ticker):
    df=pd.DataFrame(statement(ticker,'CF','A'))
    return float(df.loc['totalCashFromOperatingActivities'][0])

def ltdebt(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['longTermDebt'][0]),float(df.loc['longTermDebt'][1])
    return q-p

def current_ratio(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p=float(df.loc['totalCurrentAssets'][0])
    q=float(df.loc['totalCurrentAssets'][1])
    r=float(df.loc['totalCurrentLiabilities'][0])
    s=float(df.loc['totalCurrentLiabilities'][1])
    current_ratio1=p/r
    current_ratio2=q/s
    return current_ratio1-current_ratio2
   
def new_shares(ticker):
    df=pd.DataFrame(statement(ticker,'BS','A'))
    p,q=float(df.loc['retainedEarnings'][0]),float(df.loc['retainedEarnings'][1])
    r,s=float(df.loc['totalStockholderEquity'][0]),float(df.loc['totalStockholderEquity'][1])
    return (s-q)-(r-p)

def gross_margin(ticker):
    df=pd.DataFrame(statement(ticker,'IS','A'))
    p,q=float(df.loc['grossProfit'][0]),float(df.loc['grossProfit'][1])
    return p-q

def Asset_Turnover_Ratio(ticker):
    df1=pd.DataFrame(statement(ticker,'IS','A'))
    df2=pd.DataFrame(statement(ticker,'BS','A'))
    p,q,s=float(df2.loc['totalAssets'][0]),float(df2.loc['totalAssets'][1]),float(df2.loc['totalAssets'][2])
    av_assets1=(p+q)/2
    av_assets2=(q+s)/2
    atr1=float(df1.loc['totalRevenue'][0])/av_assets1
    atr2=float(df1.loc['totalRevenue'][1])/av_assets2
    return atr1-atr2

def Piotroski_Score(ticker):
    Pscore=0
    if net_income(ticker)>0:
        Pscore+=1
    if roa(ticker)>0:
        Pscore+=1
    if ocf(ticker)>0:
        Pscore+=1
    if ocf(ticker)>net_income(ticker):
        Pscore+=1
    #if ltdebt(ticker)>0:
    #    Pscore+=1
    if current_ratio(ticker)>0:
        Pscore+=1
    if new_shares(ticker)>0:
        Pscore+=1
    if gross_margin(ticker)>0:
        Pscore+=1
    if Asset_Turnover_Ratio(ticker)>0:
        Pscore+=1
    return Pscore

In [43]:
tickers=['INFY.NS','ABCAPITAL.NS','BATAINDIA.NS','MRF.NS','TATAPOWER.NS']
data2={}
for ticker in tickers:
    data2[ticker]=Piotroski_Score(ticker)
    data=pd.DataFrame(data2,index=[0])    
data

,INFY.NS,ABCAPITAL.NS,BATAINDIA.NS,MRF.NS,TATAPOWER.NS
0,5,7,5,7,6
